# Classes

In this assignment we will work on a more barebones version of our 'Student' class. We will try to implement some more advanced methods.

## 'Dunder' Methods

How can the ```+``` operator add numbers, concatenate strings and join lists? Because Python allows classes to _define_ what it means to add, subtract, multiply ... objects with another.   


This is usually done using methods in a class that are written between double underscores (d_under -> dunder methods), such as ```__add__```, which defines what happens when the ```+``` is invoked between two objects. In a similar way, objects may choose how they implement their 'length', e.g., the output of ```len(object)``` via their ```__len___``` method.

You should usually stick to the dunder methods as defined in standard python, and not be creative and make up your own, as it is assumed by readers that such methods interface seamlessly with standard python.

The following is an easy example how this works under the hood.

In [5]:
class MyNumber:
    def __init__(self, value):
        self.value = value

    def __add__(self, other):
        if not isinstance(other, MyNumber):
            return NotImplemented
        return MyNumber(self.value + other.value)

    def __len__(self):
        return self.value

a = MyNumber(10)
b = MyNumber(5)

# Using + operator calls __add__ under the hood
c = a + b  
d = a.__add__(b)
print(c.value)  
print(d.value)
print(c.value == d.value)

print(len(a))   
print(len(b))   


15
15
True
10
5


## Another example

Here we use a custom method to illustrate what happens 'under the hood'

In [6]:
class A:
    def __somemethod__(self):
        return "abracadabra"

class B:
    def __somemethod__(self):
        return True

class C:
    __somemethod__ = "not callable"  

class D:
    pass

def somemethod(obj):
    """
    Simulates calling a custom 'dunder' method manually, 
    like Python does under the hood.
    Normally instead of returning strings, we would throw errors!
    """
    if not hasattr(obj, "__somemethod__"):
        return (f"Object {obj.__class__.__name__} does not implement __somemethod__.")
    
    method = getattr(obj, "__somemethod__")
    if not callable(method):
        return (f"Attribute __somemethod__ of {obj.__class__.__name__} is not callable")
    
    return method()

print(somemethod(A()))
print(somemethod(B())) 
print(somemethod(D()))
somemethod(C())       


abracadabra
True
Object D does not implement __somemethod__.


'Attribute __somemethod__ of C is not callable'

## Useful Dunder Methods

There are a couple of such methods that are almost always useful to define. These let your objects behave more like built-in Python types and integrate naturally with Python’s syntax and functions. A good overview of all possible dunder methods is given here: https://www.pythonmorsels.com/every-dunder-method/.

1. **`__init__(self, …)`** – Constructor

   * Called when a new instance of the class is created.

   ```python
   class Person:
       def __init__(self, name):
           self.name = name

   p = Person("Alice")
   ```

2. **`__str__(self)`** – String representation

   * Called by `str(obj)` or `print(obj)` to get a human-readable string.

   ```python
   class Person:
       def __init__(self, name):
           self.name = name
       def __str__(self):
           return f"Person named {self.name}"

   print(Person("Alice"))  # Output: Person named Alice
   ```

3. **`__repr__(self)`** – Official string representation

   * Called by `repr(obj)` and in interactive shells. Should be unambiguous and ideally valid Python code.

   ```python
   class Person:
       def __init__(self, name):
           self.name = name
       def __repr__(self):
           return f"Person({self.name!r})"

   repr(Person("Alice"))  # Output: Person('Alice')
   ```

4. **`__eq__(self, other)`** – Equality check

   * Called by `==` to compare objects. Can also define `__lt__`, `__gt__`, etc., for ordering.

   ```python
   class Person:
       def __init__(self, name):
           self.name = name
       def __eq__(self, other):
           return self.name == other.name

   print(Person("Alice") == Person("Alice")) 
   ```


# Working on our Student Example

In our barebones student example

```python
class Student:
    def __init__(self, full_name, student_id, uni):
        """ Barebones student class """
        self.full_name = full_name
        self.student_id = student_id
        self.uni = uni
```

We have only defined some basic properties. Your task is to expand this as follows...

__Basic:__

1. Give the objects a useful integer representation, i.e. what it returns when ```int(student)``` is called.
2. Further, give them suitable string (```str()```, ```repr()```) representations.
3. Define when two student objects are the same, i.e. the behaviour of ```student1 == student2```

__Advanced:__

4. Define how Python should compare two students using `<`, `<=`, `>`, `>=` operators. For example, you could base ordering on `student_id` or `full_name`.*Hint: Implement `__lt__` (less than), `__le__` (less or equal), `__gt__` (greater than), and `__ge__` (greater or equal).*

```python
student1 < student2  
```

5. To be used as keys in dictionaries or elements in sets, objects need to be hashable. A hash value is an integer that uniquely (as much as possible) represents the object and is returned by the object’s __hash__() method. Objects that should be considered the same should have the same hash.  *Hint: Implement `__hash__` consistent with `__eq__`*.

```python
studentgroup = {student1: "Math 101", student2: "Physics 202"}
```


In [13]:
# Implement the dunder methods here
class Student:
    def __init__(self, full_name, student_id, uni):
        """ Barebones student class """
        self.full_name = full_name
        self.student_id = student_id
        self.uni = uni

In [12]:
class Student:
    def __init__(self, full_name, student_id, uni):
        """ Barebones student class """
        self.full_name = full_name
        self.student_id = student_id
        self.uni = uni


In [8]:
def test_student_class():
    # --- Create some students ---
    s1 = Student("Alice", 101, "UniX")
    s2 = Student("Bob", 102, "UniX")
    s3 = Student("Alice", 101, "UniX")  # Same as s1
    s4 = Student("Charlie", 101, "UniY")  # Same ID as s1 but different uni

    # --- Test int representation ---
    assert int(s1) == 101, "int() failed"

    # --- Test equality ---
    assert s1 == s3, "Equality (__eq__) failed"
    assert s1 != s2, "Inequality failed"
    assert s1 != s4, "Equality should consider university"

    # --- Test ordering ---
    assert s1 < s2, "__lt__ failed"
    assert s2 > s1, "__gt__ failed"
    assert s1 <= s3, "__le__ failed for equal objects"
    assert s1 >= s3, "__ge__ failed for equal objects"

    # --- Test hashing ---
    student_set = {s1, s2, s3}
    assert len(student_set) == 2, "Hashing failed" # since s3 == s1, the set should have only 2 elements

    # --- Test using students as dictionary keys ---
    student_dict = {s1: "Python for Neuroscience", s2: "MATLAB for Engineering"}
    assert student_dict[s1] == "Python for Neuroscience", "Dict key lookup failed"

    print("All tests passed!")

test_student_class()


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Student'


## Introduction to Properties, Getters, and Setters

In Python, a **property** allows controlled access to an object's attributes. Using **getters** and **setters**, you can:

* **Control read/write access** to attributes.
* **Validate data** before setting it.
* Make an attribute **read-only** if needed.
* Define attributes dynamical, e.g., update a circles circumference attribute if the diameter changes.

Python provides a decorator-based syntax:

* `@property` → defines a **getter** (access like a normal attribute).
* `@<property_name>.setter` → defines a **setter** (called when assigning a value).

**Example**

```python
class Example:
    def __init__(self, value):
        self._value = value  # internal variable

    @property
    def value(self):
        # This is the getter
        return self._value

    @value.setter
    def value(self, new_value):
        # This is the setter
        if new_value < 0:
            raise ValueError("Value must be non-negative")
        self._value = new_value
```


## Implementing properties

You can now try to implement some properties with these more advanced access methods.

### Beginning

For simplicity, we will again start with our basic 'Student' skeleton.

```python
class Student:
    def __init__(self, full_name, student_id, uni):
        self.full_name = full_name
        self.student_id = student_id
        self.uni = uni
```

### Your Task

We now want to add some input validation and access management. For that, try to use the property and setter logic to make the class fulfill the following constraints.

1. **`full_name` property**

   * Must be a string.
   * Must have at least 3 characters.

2. **`student_id` property**

   * Must be an integer.
   * Must be positive.

3. **`uni` property**

   * Must be a string.
   * Optional: Only allow certain universities (e.g., `"UniX"`, `"UniY"`).

### Testing

In the end, your code should succeed in these cases:

```python
s = Student("Alice", 101, "UniX")

s.full_name = "Bob"
s.student_id = 202
s.uni = "UniY"
```

and fail for these cases:

```python
s.full_name = "zy"
s.student_id = -5
s.uni = None
```


In [9]:
# Expand this Class using properties and setters!

class Student:
    def __init__(self, full_name, student_id, uni):
        self.full_name = full_name
        self.student_id = student_id
        self.uni = uni

In [10]:
# After implementing and running the class above, test it here!
s = Student("Alice", 101, "UniX")
s.full_name = "Bob"
s.student_id = 202
s.uni = "UniY"
print(s.full_name, s.student_id, s.uni) 

# For the following, we should get error messages!

try:
    s.full_name = "zy"
    print("Full Name Check not passed.")
except ValueError as e:
    print(e)  # Full name must be at least 3 characters

try:
    s.student_id = -5
    print("Student ID Check not passed.")
except ValueError as e:
    print(e)  # Student ID must be positive

try:
    s.uni = None
    print("None University Check not passed.")
except ValueError as e:
    print(e)  # University must be a string

try:
    s.uni = "RandomUniversityofNothing"
    print("Outside University Check not passed.")
except ValueError as e:
    print(e)  


Bob 202 UniY
Full Name Check not passed.
Student ID Check not passed.
None University Check not passed.
Outside University Check not passed.
